# CEOS Data Cube - Fractional Coverage Analysis Notebook
*****
**Description:** This Python notebook calculates vegetation fractional cover using a Landsat-7 Data Cube. Vegetation fractional cover represents the exposed proportion of Photosynthetic Vegetation (PV), Non-Photosynthetic Vegetation (NPV) and Bare Soil (BS) within each pixel. This product is useful for natural resource management, modeling carbon dynamics and assessing land cover change in time series. Accurate estimation of fractional cover is especially important for monitoring and modeling savanna systems, subject to highly seasonal rainfall and drought, grazing by domestic and native animals, and frequent burning. 
*****
**Import necessary Data Cube libraries and dependencies.**

In [1]:
import ipywidgets as widgets

In [2]:
%matplotlib inline

from datetime import datetime
import numpy as np
import xarray as xr
import scipy.optimize as opt 

import datacube
import utils.data_cube_utilities.dc_utilities as utilities
from utils.data_cube_utilities.dc_mosaic import create_mean_mosaic, ls7_unpack_qa

from dc_notebook_utilities import create_platform_product_gui, generate_metadata_report, create_extents_gui
from utils.data_cube_utilities.dc_display_map import display_map

'Basemap' was not found in 'mpl_toolkits.basemap'.  It is likely that 'mpl_toolkits.basemap' is not present


**First, we must connect to our data cube.** We can then query the contents of the data cube we have connected to, including both the metadata and the actual data.

In [3]:
dc = datacube.Datacube(config='/home/localuser/.datacube.conf', app='dc-frac-cov-analysis')
api = datacube.api.API(datacube=dc)

**Obtain the metadata of our cube.** Initially, we need to get the platforms and products in the cube. The rest of the metadata will be dependent on these two options.

In [4]:
# Get available products
products = dc.list_products()
platform_names = list(set(products.platform))
product_names = list(products.name)

**Execute the following code and then use the generated form to choose your desired platfrom and product.**

In [5]:
product_values = create_platform_product_gui(platform_names, product_names, dc, 
                                             default_platform='LANDSAT_7', default_product="ls7_ledaps_colombia")

interactive(children=(Select(description='platform', index=6, options=('TERRA', 'GPM', 'SENTINEL_1', 'LANDSAT_5', 'ALOS_2', 'ALOS', 'LANDSAT_7', 'LANDSAT_8'), value='LANDSAT_7'), Output()), _dom_classes=('widget-interact',))

In [6]:
# Save the form values
platform = product_values[0][0]
product = product_values[1][0]

# Get the pixel resolution of the selected product
resolution = products.resolution[products.name.values == product]
lat_dist = resolution.values[0][0]
lon_dist = resolution.values[0][1]

# Get the extents of the cube
descriptor = api.get_descriptor({'platform': platform})[product]

min_date = descriptor['result_min'][0]
min_lat = descriptor['result_min'][1]
min_lon = descriptor['result_min'][2]

min_date_str = str(min_date.year) + '-' + str(min_date.month).zfill(2) + '-' + str(min_date.day).zfill(2)

min_lat_rounded = round(min_lat, 3)
min_lon_rounded =  round(min_lon, 3)

max_date = descriptor['result_max'][0]
max_lat = descriptor['result_max'][1] 
max_lon = descriptor['result_max'][2] 

max_date_str = str(max_date.year) + '-' + str(max_date.month).zfill(2) + '-' + str(max_date.day).zfill(2)

max_lat_rounded = round(max_lat, 3)
max_lon_rounded = round(max_lon, 3)

# Display metadata
generate_metadata_report(min_date_str, max_date_str, 
                         min_lon_rounded, max_lon_rounded, lon_dist,
                         min_lat_rounded, max_lat_rounded, lat_dist)

,Min,Max,Resolution
Date:,2000-01-03,2015-12-14,
Longitude:,-76.518,-74.046,0.000269505
Latitude:,-0.946,2.402,-0.000271307


In [7]:
# Reduce the area for faster processing times.
# mid_lon = (min_lon + max_lon) / 2
# min_lon_small = (min_lon + mid_lon) / 2
# min_lon_small_rounded = round(min_lon_small, 3)
# max_lon_small = (mid_lon + max_lon) / 2
# max_lon_small_rounded = round(max_lon_small, 3)
# mid_lat = (min_lat + max_lat) / 2
# min_lat_small = (min_lat + mid_lat) / 2
# min_lat_small_rounded = round(min_lat_small, 3)
# max_lat_small = (mid_lat + max_lat) / 2
# max_lat_small_rounded = round(max_lat_small,3)
# lat_small = (min_lat_small_rounded, max_lat_small_rounded)
# lon_small = (min_lon_small_rounded, max_lon_small_rounded)
# display_map(lat_small, lon_small)
min_lat_small = 0.9300
min_lat_small_rounded = round(min_lat_small, 3)
max_lat_small = 1.0443
max_lat_small_rounded = round(max_lat_small, 3)
min_lon_small = -74.7894
min_lon_small_rounded = round(min_lon_small, 3)
max_lon_small = -74.6679
max_lon_small_rounded = round(max_lon_small, 3)
lat_small = (min_lat_small, max_lat_small)
lon_small = (min_lon_small, max_lon_small)
display_map(lat_small, lon_small)
# display_map((min_lat, max_lat), (min_lon, max_lon))

**Execute the following code and then use the generated form to choose the extents of your desired data.**

In [8]:
# option_selected = widgets.Select(options=platforms, value=default_platform)
# init = option_selected.value
# init
extent_values = create_extents_gui(min_date_str, max_date_str,
                                   min_lon_small_rounded, max_lon_small_rounded,
                                   min_lat_small_rounded, max_lat_small_rounded)

Label(value='Start date: ')

Text(value='2000-01-03')

Label(value='End date: ')

Text(value='2015-12-14')

Label(value='Min lon: ')

...

BoundedFloatText(value=-74.668, max=-74.668, min=-74.789)

Label(value='Min lat: ')

BoundedFloatText(value=0.93, max=1.044, min=0.93)

Label(value='Max lat: ')

BoundedFloatText(value=1.044, max=1.044, min=0.93)

**Now that we have filled out the above two forms, we have enough information to query our data cube.** The following code snippet ends with the actual Data Cube query, which will return the dataset with all the data matching our query.

In [9]:
# Save form values
start_date = datetime.strptime(extent_values[0].value, '%Y-%m-%d')
end_date = datetime.strptime(extent_values[1].value, '%Y-%m-%d')
min_lon = extent_values[2].value
max_lon = extent_values[3].value
min_lat = extent_values[4].value
max_lat = extent_values[5].value

In [11]:
# Query the Data Cube
dataset_in = dc.load(platform=platform,
                     product=product,
                     time=(start_date, end_date),
                     lon=(min_lon_small, max_lon_small), 
                     lat=(min_lat_small, max_lat_small), measurements=['red', 'green', 'blue','nir','swir1', 'swir2',  'pixel_qa'])

KeyboardInterrupt: 

**At this point, we have finished accessing our data cube and we can turn to analyzing our data.** In this example, we will run the fractional coverage algorithm as presented in Guerschman.
*****
For more information on the fractional coverage algorithm, refer to:
Guerschman, Juan P., et al. (2015) "Assessing the effects of site heterogeneity and soil properties when unmixing photosynthetic vegetation, non-photosynthetic vegetation and bare soil fractions from Landsat and MODIS data." Remote Sensing of Environment 161: 12-26.
*****
**Create a cloudfree, most-recent pixel mosaic from the retrieved dataset.** The fractional coverage algorithm will then be applied to the mosaic.

In [ ]:

clean_mask  = ls7_unpack_qa(dataset_in.pixel_qa, "clear").astype(bool)

mosaic = create_mean_mosaic(dataset_in, clean_mask=clean_mask)


Clean and format data so it is analysis-ready.

In [ ]:
band_stack = []

for band in [mosaic.blue.values, mosaic.green.values, mosaic.red.values,
             mosaic.nir.values, mosaic.swir1.values, mosaic.swir2.values]:
    band = band.astype(np.float32)
    band = band * 0.0001 
    band = band.flatten()
    band_clean = np.full(band.shape, np.nan)
    band_clean[mosaic_clean_mask] = band[mosaic_clean_mask]
    band_stack.append(band_clean)
    
band_stack = np.array(band_stack).transpose()

In order to account for the non-linearities in the spectral mixing, the following code performs log transformations of the surface reflectance bands and interactive terms in the regression equations.

In [ ]:
for b in range(6):
    band_stack = np.hstack((band_stack, np.expand_dims(np.log(band_stack[:, b]), axis=1)))
for b in range(6):
    band_stack = np.hstack((band_stack, np.expand_dims(np.multiply(band_stack[:, b], band_stack[:, b+6]), axis=1)))
for b in range(6):
    for b2 in range(b+1, 6):
        band_stack = np.hstack((band_stack, np.expand_dims(np.multiply(band_stack[:, b], band_stack[:, b2]), axis=1))) 
for b in range(6):
    for b2 in range(b+1, 6):
        band_stack = np.hstack((band_stack, np.expand_dims(np.multiply(band_stack[:, b+6], band_stack[:, b2+6]), axis=1)))
for b in range(6):
    for b2 in range(b+1, 6):
        band_stack = np.hstack((band_stack, 
                                np.expand_dims(np.divide(band_stack[:, b2] - band_stack[:, b], 
                                                         band_stack[:, b2] + band_stack[:, b]), 
                                               axis=1)))

band_stack = np.nan_to_num(band_stack)  # Now an n x 63 matrix

Run fractional coverage algorithm.

In [ ]:
ones = np.ones(band_stack.shape[0])
ones = ones.reshape(ones.shape[0], 1)
band_stack = np.concatenate((band_stack, ones), axis=1) # Now an n x 64 matrix

end_members = np.loadtxt('utils/endmembers_landsat.csv', delimiter=',') # Creates a 64 x 3 matrix

SumToOneWeight = 0.02
ones = np.ones(end_members.shape[1]) * SumToOneWeight
ones = ones.reshape(1, end_members.shape[1])
end_members = np.concatenate((end_members, ones), axis=0).astype(np.float64)

result = np.zeros((band_stack.shape[0], end_members.shape[1]), dtype=np.float64) # Creates an n x 3 matrix

for i in range(band_stack.shape[0]):
    if mosaic_clean_mask[i]:
        result[i, :] = (opt.nnls(end_members, band_stack[i, :])[0].clip(0, 2.54)*100).astype(np.int16)
    else:
        result[i, :] = np.ones((end_members.shape[1]), dtype=np.int16)*(255)

result = result.reshape(mosaic.latitude.size, mosaic.longitude.size, 3)

**The following plot visualizes the results of the fractional coverage algorithm.**

In [ ]:
result